In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Dropout, merge
from keras.layers import LSTM,Bidirectional,GRU,SimpleRNN
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D, Input

import tensorflow as tf
from keras.layers import Conv1D, GlobalMaxPooling1D,GlobalAveragePooling1D,MaxPooling1D, AveragePooling1D
from scipy.stats import pearsonr
from sklearn.cross_validation import train_test_split

Using Theano backend.


In [2]:
import keras.backend as K

def pearson(X,Y):
    #return K.mean(K.abs(y_true-y_pred))
    if K.sum(X)==0:return 0
    n = K.sum(X)/K.mean(X)
    if n==0:return 0
    num = K.sum(X*Y) - K.sum(X)*K.sum(Y)/n
    den = (K.sum(X*X) - K.square(K.sum(X))/n)*(K.sum(Y*Y) - K.square(K.sum(Y))/n)
    den = K.sqrt(den)
    if den==0 : return 0
    return num/den

def pearson1(X,Y):
    #return K.mean(K.abs(y_true-y_pred))
    if K.sum(X)==0:return 0
    n = K.sum(X)/K.mean(X)
    if n==0:return 0
    num = K.sum(X*Y) - K.sum(X)*K.sum(Y)/n
    den = (K.sum(X*X) - K.square(K.sum(X))/n)*(K.sum(Y*Y) - K.square(K.sum(Y))/n)
    den = K.sqrt(den)
    if den==0 : return 0
    return -1 * num/den

# prepare train, dev, test data

In [3]:
emotions = ['fear']
x_train_1, y_train, x_test_1, y_test, x_train_2, x_test_2 = {},{},{},{},{},{}
for emotion in emotions : 
    
    x1t = np.load('../intermediates/train/'+emotion+'/X_'+emotion+'2.npy')
    x2t = np.load('../intermediates/train/'+emotion+'/lexicon.npy')
    x3t = np.load('../intermediates/train/'+emotion+'/train_'+emotion+'2.npy')
    yt = np.load('../intermediates/train/'+emotion+'/labels.npy')
    x1d = np.load('../intermediates/val/'+emotion+'/X_'+emotion+'_dev2.npy')
    x2d = np.load('../intermediates/val/'+emotion+'/lexicon.npy')
    x3d = np.load('../intermediates/val/'+emotion+'/dev_'+emotion+'2.npy')
    yd = np.load('../intermediates/val/'+emotion+'/labels.npy')    
    x1te = np.load('../intermediates/test/'+emotion+'/X_'+emotion+'2.npy')
    x2te = np.load('../intermediates/test/'+emotion+'/lexicon.npy')
    x3te = np.load('../intermediates/test/'+emotion+'/test_'+emotion+'2.npy')
    yte = np.load('../intermediates/test/'+emotion+'/labels.npy')
    
    x1t = np.concatenate([x1t,x2t], axis=1)
    x1d = np.concatenate([x1d,x2d], axis=1)
    x1te = np.concatenate([x1te,x2te], axis=1)
    
    x1t = np.concatenate([x1t,x1d], axis=0)
    x3t = np.concatenate([x3t,x3d], axis=0)
    yt = np.concatenate([yt,yd], axis=0)
    
    x_train_1[emotion] = x1t
    y_train[emotion] = yt
    x_test_1[emotion] = x1te
    y_test[emotion] = yte
    x_train_2[emotion] = x3t
    x_test_2[emotion] = x3te
    print emotion + ': ', x1t.shape, yt.shape, x1te.shape, yte.shape, x3t.shape, x3te.shape
    

fear:  (1257, 243) (1257,) (995, 243) (995,) (1257, 50, 200) (995, 50, 200)


# ensemble of feed-forward NN and CNN/LSTM for each emotion

## anger

In [5]:
ypred = np.zeros(995)
for i in range(7) : 
    input1 = Input(shape=(243,))
    l1 = Dense(150, init='normal', activation='relu')(input1)
    #l1 = Dropout(p=0.2)(l1)
    l1 = Dense(75, init='normal', activation='relu')(l1)
    #l1 = Dense(25, init='normal', activation='relu')(l1)
    #l1 = Dropout(p=0.2)(l1)
    l1 = Dense(25, init='normal', activation='relu')(l1)
    #o1 = Dense(1, init='normal')(l1)

    #cnn-lstm-1
    input2 = Input(shape=(50,200,))
    l2 = Conv1D(125,3,activation='relu')(input2)
    l2 = GlobalMaxPooling1D()(l2)
    #l2 = Dropout(p=0.3)(l2)
    l2 = Dense(75,activation='relu')(l2)
    l2 = Dense(25,activation='relu')(l2)
    #l2 = Dense(25,activation='relu')(l2)
    #o2 = Dense(1)(l2)

    merged_output = merge([l1, l2], mode='concat', concat_axis=-1)
    merged_output = Dense(25, activation='relu')(merged_output)
    merged_output = Dropout(p=0.2)(merged_output)
    predictions = Dense(1, activation='sigmoid')(merged_output)

    model = Model(input=[input1, input2], output=predictions)
    model.compile(loss='mae', optimizer='adam', metrics=[pearson])
    model.fit([x_train_1['fear'], x_train_2['fear']], y_train['fear'], 
              nb_epoch=20, verbose=2, batch_size=8,
             )
    tmp = model.predict([x_test_1['fear'], x_test_2['fear']])
    print pearsonr(tmp[:,0], y_test['fear'])
    ypred += tmp[:,0]
print pearsonr(ypred/7.0, y_test['fear'])
np.save('fear1_glove.npy', ypred/7.0)

Epoch 1/20
3s - loss: 0.1409 - pearson: nan
Epoch 2/20
2s - loss: 0.1135 - pearson: nan
Epoch 3/20
2s - loss: 0.0979 - pearson: nan
Epoch 4/20
2s - loss: 0.0872 - pearson: nan
Epoch 5/20
2s - loss: 0.0793 - pearson: nan
Epoch 6/20
2s - loss: 0.0743 - pearson: nan
Epoch 7/20
2s - loss: 0.0699 - pearson: nan
Epoch 8/20
2s - loss: 0.0622 - pearson: nan
Epoch 9/20
2s - loss: 0.0573 - pearson: nan
Epoch 10/20
2s - loss: 0.0543 - pearson: nan
Epoch 11/20
2s - loss: 0.0543 - pearson: nan
Epoch 12/20
2s - loss: 0.0497 - pearson: nan
Epoch 13/20
2s - loss: 0.0507 - pearson: nan
Epoch 14/20
2s - loss: 0.0475 - pearson: nan
Epoch 15/20
2s - loss: 0.0464 - pearson: nan
Epoch 16/20
2s - loss: 0.0434 - pearson: nan
Epoch 17/20
2s - loss: 0.0464 - pearson: nan
Epoch 18/20
3s - loss: 0.0416 - pearson: nan
Epoch 19/20
4s - loss: 0.0421 - pearson: nan
Epoch 20/20
3s - loss: 0.0397 - pearson: nan
(0.73188040752382499, 1.3421489818501614e-167)
Epoch 1/20
3s - loss: 0.1463 - pearson: nan
Epoch 2/20
4s - lo

## fear 